## CPI 结果与分析
以双十一前后一周的价格为数据 我们计算了以11月5日为基础的价格指数，并绘图如下：

### 总CPI
![img](https://github.com/codeandworld/repo_photo/blob/master/double11_totalindex.png?raw=true)
- 价格除了双十一的的那一天，价格指数基本大于一，整体呈现上升趋势。
- 双十一的价格急剧下降。
- 双十一过后的第二天价格指数并未出现剧烈上升，说明部分商品在双十一过后折扣仍然存在。

### 八大类CPI
![img](https://github.com/codeandworld/repo_photo/blob/master/double11_8cate.png?raw=true)
- 从八个大类的价格指数来看，整体和总的价格指数波动相近，但是也存在部分的差异。
- 衣服的价格下降最为剧烈，其次是医疗产品，交通和通信产品。食品的价格也有剧烈的下降。
- 烟酒的价格指数非常奇怪，有待进一步探讨。

### 衣服中类CPI
![img](https://github.com/codeandworld/repo_photo/blob/master/double11_data_mediumindex.png?raw=true)
- 中类包含了衣服和鞋帽，从图中可以看出，衣服的价格下降比鞋帽的更为剧烈。

### 衣服小类CPI
![img](https://github.com/codeandworld/repo_photo/blob/master/double11_data_addindex.png?raw=true)
- 我们把衣服小类的CPI绘图，可以看出来，女装的价格下降最明显，其次是男装，然后是儿童。

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 03 14:41:00 2016

@author: Administrator
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# read document as dataframe
table = pd.read_csv(r'C:\Users\Administrator\Desktop\double11\our')


# get basket of goods
table_basket = table.iloc[:,[1,2,3,6]]
table_basket = table_basket[table_basket['date']=='2016-11-05']
table_basket = table_basket.iloc[:,[0,2,3]]
table_basket = table_basket.groupby(['catid', 'goodsID'], as_index=False).mean()
table_basket = table_basket.drop_duplicates(subset='goodsID')
table_basket = table_basket.set_index(['goodsID'])


# get price of goods
table_price = table.iloc[:,[2,3,7]]
table_price = table_price.groupby(['date', 'goodsID'], as_index=False).mean()
table_price = pd.pivot_table(table_price, index=['goodsID'], values=['price'], columns=['date'])
table_price.columns = ['11-04','11-05',
                        '11-06','11-07','11-08',
                        '11-09','11-10','11-11',
                        '11-12','11-13','11-14',
                        '11-15','11-16']

                        
# connect price and basket of goods
price_change = pd.concat([table_basket, table_price], axis=1, join='inner')
price_change = price_change.drop(['11-04','11-14'], 1) 


# get small category mean price
small_total = price_change.groupby(['catid'], as_index=False).mean()
small_total['medium_catid'] = small_total['catid'].apply(lambda x: str(x)[0:6])
# get medium category mean price
medium_total = small_total.groupby(['medium_catid'], as_index=False).mean()
medium_total['big_catid'] = medium_total['medium_catid'].apply(lambda x: str(x)[0:4])
# get big category mean price
big_total = medium_total.groupby(['big_catid'], as_index=False).mean()


# get index of each kind of categories
def index(qp_total):
    i_total = qp_total.copy()
    i_total['11-05'] = qp_total['11-05']/qp_total['11-05']
    i_total['11-06'] = qp_total['11-06']/qp_total['11-05']
    i_total['11-07'] = qp_total['11-07']/qp_total['11-06']
    i_total['11-08'] = qp_total['11-08']/qp_total['11-07']
    i_total['11-09'] = qp_total['11-09']/qp_total['11-08']
    i_total['11-10'] = qp_total['11-10']/qp_total['11-09']
    i_total['11-11'] = qp_total['11-11']/qp_total['11-10']
    i_total['11-12'] = qp_total['11-12']/qp_total['11-11']
    i_total['11-13'] = qp_total['11-13']/qp_total['11-12']
    i_total['11-15'] = qp_total['11-15']/qp_total['11-13']
    i_total['11-16'] = qp_total['11-16']/qp_total['11-15']
    return i_total
# smallindex
smallindex = index(small_total)
smallindex = smallindex.drop(['monthly_sales','medium_catid'], 1) 
smallindex = smallindex.set_index(['catid'])
# medium index
mediumindex = index(medium_total)
mediumindex = mediumindex.drop(['monthly_sales','catid','big_catid'], 1) 
mediumindex = mediumindex.set_index(['medium_catid'])
# bigindex
bigindex = index(big_total)
bigindex = bigindex.drop(['monthly_sales','catid'], 1) 
bigindex = bigindex.set_index(['big_catid'])
# total index
total_basket = pd.DataFrame({'weight':[0.3139, 0.0399, 0.0851, 0.0584, 
                0.0904, 0.0925, 0.1415, 0.1782]}, 
                index = ['1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108'])
total_basket = pd.concat([total_basket, bigindex], axis=1)
for i in range(1, 12):
    total_basket.iloc[:,i] = total_basket['weight']*total_basket.iloc[:,i]
total_basket = total_basket.drop(['weight'], 1) 
total_basket = total_basket.T
total_basket['total_index'] = total_basket.apply(lambda x: x.sum(), axis=1)
total_index = total_basket['total_index']


# plot totalindex
total_index.plot()
plt.savefig(r'C:\Users\Administrator\Desktop\double11_data_bingindex.png')
# plot bigindex
bigindex = bigindex.T
fig, axes = plt.subplots(nrows=4, ncols=2, sharex=True, sharey=True)
bigindex['1101'].plot(ax=axes[0,0]); axes[0,0].set_title('food')
bigindex['1102'].plot(ax=axes[0,1]); axes[0,1].set_title('tobacco and liquor')
bigindex['1103'].plot(ax=axes[1,0]); axes[1,0].set_title('clothing')
bigindex['1104'].plot(ax=axes[1,1]); axes[1,1].set_title('household')
bigindex['1105'].plot(ax=axes[2,0]); axes[2,0].set_title('healthcare')
bigindex['1106'].plot(ax=axes[2,1]); axes[2,1].set_title('transportation and communication')
bigindex['1107'].plot(ax=axes[3,0]); axes[3,0].set_title('education and recreation')
bigindex['1108'].plot(ax=axes[3,1]); axes[3,1].set_title('residence')
plt.savefig(r'C:\Users\Administrator\Desktop\double11_data_bingindex.png')